**MI6 Debrief:**

> Should be have a flip flag for each metric? To identify good/bad *Solved, negative coefficients will account for that*

> Should we fill 0 to NA data? or fill it up by mean? *Used both*

> Missing data is an issue, regression wont run unless complete data

> Should we remove 2020 data from control variables *Have done, data only upto 2019 now*

> Have not included cuurency data yet, was not clear.*Phase 2*

> Inflation cleaned .xlsx is the latest version right?

**007, I'm your new Quartermaster.**

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from functools import reduce
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from scipy.stats import yeojohnson
from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import acorr_breusch_godfrey as bg

**What makes you think this is my first time?**

In [3]:
import warnings
warnings.filterwarnings("ignore")

**A gun and a radio...not exactly Christmas, is it?**

In [4]:
Dp_FDI_wide = pd.read_csv("Dp_FDI.csv")
#Dp_FDI_wide = pd.read_csv("FDI_yoy.csv")
Dp_PE_wide = pd.read_csv("Dp_PENew.csv")
Dp_Curr_wide = pd.read_csv("Dp_Curr.csv")
Dp_IntRate_wide = pd.read_csv("Dp_IntRate.csv")
Ind_TerrIdx = pd.read_csv("Ind_TerrIdx.csv")
Ind_TerrRaw = pd.read_csv("Ind_Terr.csv")
Cn_LaborForce_wide = pd.read_csv("Cn_LaborForce.csv")
Cn_GDPpc_wide = pd.read_csv("Cn_GDPpc.csv")
Cn_EODB_wide = pd.read_csv("EODBPercentiled.csv")
Cn_DomCrdt_wide = pd.read_csv("Cn_DomCrdt.csv")
Cn_FixedCapForm_wide = pd.read_csv("Cn_FixedCapForm.csv")
Cn_Infl_wide = pd.read_csv("Cn_Infl.csv")
Cn_IntRateL_wide = pd.read_csv("Cn_IntRateL.csv")
Cn_CurrStd_wide = pd.read_csv("Cn_CurrStd.csv")

In [5]:
Dp_FDI = pd.melt(Dp_FDI_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "FDI")
Dp_PE = pd.melt(Dp_PE_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "PE")
Dp_Curr = pd.melt(Dp_Curr_wide,id_vars = ("Year"), var_name = "Country Name", value_name = "Currency")
Dp_IntRate = pd.melt(Dp_IntRate_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "IntRate")
Cn_LaborForce = pd.melt(Cn_LaborForce_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "Lab")
Cn_GDPpc = pd.melt(Cn_GDPpc_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "GDPpc")
Cn_EODB = pd.melt(Cn_EODB_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "EODB")
Cn_DomCrdt = pd.melt(Cn_DomCrdt_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "DomCrdt")
Cn_FixedCapForm = pd.melt(Cn_FixedCapForm_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "FixedCapForm")
Cn_Infl = pd.melt(Cn_Infl_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "Inf")
Cn_IntRateL = pd.melt(Cn_IntRateL_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "IntRateLend")
Cn_CurrStd = pd.melt(Cn_CurrStd_wide,id_vars = ("Year"), var_name = "Country Name", value_name = "CurStd")[["Country Name", "Year", "CurStd"]]
Cn_CurrStd["Year"] = Cn_CurrStd["Year"].apply(str)

In [6]:
# Merging control variables into one dataset
dfs = [Cn_LaborForce, Cn_GDPpc, Cn_EODB, Cn_DomCrdt,Cn_FixedCapForm, Cn_Infl, Cn_IntRateL,Cn_CurrStd]
Control_Matrix = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                        left_on=["Country Name","Year"], right_on = ["Country Name","Year"], how='outer'), dfs)
Control_Matrix["Year"] = pd.to_numeric(Control_Matrix["Year"])
Control_Matrix = Control_Matrix[Control_Matrix["Year"]<2020]
Control_Matrix["Year"] = Control_Matrix["Year"].apply(str)

In [7]:
#1a. Terrorism Incidents Data as many gaps. So creating dummy '0' datapoints for completeness
Ind_Terr_Wide = Ind_TerrRaw.pivot(index="Year", columns="Country Name", values= "Incidents").reset_index(drop=False).fillna(0)
Ind_Terr = pd.melt(Ind_Terr_Wide,id_vars = ("Year"), var_name = "Country Name", value_name = "Incidents")

#1b. Including Terrorism Incidents Lag Variables
for i in range(0,len(Ind_Terr["Country Name"].unique())):
    country = Ind_Terr["Country Name"].unique()[i]
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    Ter["IncidentLag_5"] = Ter["Incidents"].shift(5)
    if i == 0:
        Ind_Terr_Ref = Ter
    else:
        Ind_Terr_Ref = pd.concat([Ind_Terr_Ref, Ter])
#Ind_Terr_Ref.dropna(inplace=True)

#2a. Including Terrorism Index Lag Variables
for i in range(0,len(Ind_TerrIdx["Country Name"].unique())):
    country = Ind_TerrIdx["Country Name"].unique()[i]
    Ter = Ind_TerrIdx[Ind_Terr["Country Name"] == country]
    Ter["TerrorIdxLag_5"] = Ter["TerrIdx"].shift(5)
    if i == 0:
        Ind_TerrIdx_Ref = Ter
    else:
        Ind_TerrIdx_Ref = pd.concat([Ind_TerrIdx_Ref, Ter])
#Ind_TerrIdx_Ref.dropna(inplace=True)

**Be careful what you wish for.**

In [8]:
#Replcaing Nan values in Control Matrix with mean()
for i in range(0,len(Control_Matrix["Country Name"].unique())):
    country = Control_Matrix["Country Name"].unique()[i]
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt","FixedCapForm", "Inf", "IntRateLend"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt","FixedCapForm", "Inf", "IntRateLend"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    if i == 0:
        Control_Matrix_Ref = Cntrl
    else:
        Control_Matrix_Ref = pd.concat([Control_Matrix_Ref, Cntrl])
#Control_Matrix_Ref.dropna(inplace=True)

In [107]:
#Combine and export database to check
Ind_Terr_Ref["Year"] = Ind_Terr_Ref["Year"].apply(str)
Ind_TerrIdx_Ref["Year"] = Ind_TerrIdx_Ref["Year"].apply(str)
Dp_Curr["Year"] = Dp_Curr["Year"].apply(str)
Dp_PE["Year"] = Dp_PE["Year"].apply(str)
df2 = [Dp_FDI, Dp_PE, Dp_Curr, Dp_IntRate, Ind_Terr_Ref, Ind_TerrIdx_Ref, Control_Matrix_Ref]
#df2 = [Dp_IntRate, Ind_Terr_Ref, Ind_TerrIdx_Ref, Control_Matrix_Ref]
MergedFileRaw = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                        left_on=["Country Name","Year"], right_on = ["Country Name","Year"], how='outer'), df2)
print("Before dropping NA",MergedFile.shape)
print("After dropping NA",MergedFile.shape)

Before dropping NA (406, 13)
After dropping NA (406, 13)


In [11]:
addvalue = abs(int(min(list(MergedFile.min()[2:]))/5 - 1)*5)

In [ ]:
transform_type = ""

In [ ]:
if transform_type == "sqrt":
    MergedFile.iloc[:,2:] = MergedFile.iloc[:,2:]+addvalue
    MergedFile.iloc[:,2:] = np.sqrt(MergedFile.iloc[:,2:])
elif transform_type == "log":
    MergedFile.iloc[:,2:] = MergedFile.iloc[:,2:]+addvalue
    MergedFile.iloc[:,2:] = np.log(MergedFile.iloc[:,2:])

countries = set(Ind_Terr["Country Name"].unique()).intersection(Control_Matrix["Country Name"].unique()).intersection(Dp_FDI["Country Name"].unique())

In [108]:
qwe = [0,1,5,8,7,10,11,12,13,14,15,16,17]

In [110]:
MergedFile

,Country Name,Year,IntRate,TerrIdx,IncidentLag_5,Lab,GDPpc,EODB,DomCrdt,FixedCapForm,Inf,IntRateLend,CurStd
93,Australia,2001,2.122744,0.115,5.0,63.56,0.565689,9.571429,88.651827,23.279665,4.620835,6.841667,0.221192
94,Bolivia,2001,17.853099,0.000,1.0,71.19,-0.235740,1.285714,49.293750,13.926773,1.870093,20.057061,0.538276
95,Canada,2001,3.692988,1.140,1.0,65.37,0.309870,9.428571,121.352832,20.151765,2.044026,5.812500,0.058173
96,Dominican Republic,2001,17.184522,0.000,3.0,58.37,0.942845,3.785714,23.570357,23.578260,6.040739,24.263333,0.857944
99,Guatemala,2001,26.027845,1.033,26.0,62.22,-0.096568,3.857143,22.766713,18.601937,-5.610145,18.957500,0.812949
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826,Ukraine,2019,10.692914,4.692,896.0,54.16,3.815825,5.000000,30.017994,17.610520,8.246955,19.821708,1.994713
1831,Tajikistan,2019,19.188959,4.180,1.0,39.68,4.863057,2.000000,11.582367,31.508865,3.658684,23.549707,1.311922
1832,Vietnam,2019,5.814896,0.420,0.0,77.37,6.001037,6.000000,137.912126,24.232395,1.788599,7.707500,510.257117
1833,Belarus,2019,0.365104,0.000,0.0,64.06,2.106840,7.000000,28.768881,27.034537,8.626733,9.023333,0.186606


In [109]:
MergedFile = MergedFileRaw.iloc[:,qwe].dropna()

In [ ]:
C:\Users\aniro\Desktop\qf603_Quant\Project\Phinal\Results1\FDI\CountryWise

In [111]:
path = "C:\\Users\\aniro\\Desktop\\qf603_Quant\\Project\\Phinal\\Results1\\Int\\CountryWise"

In [112]:
import os
os.chdir(path)

In [113]:
for country in MergedFile["Country Name"].unique():
    print("Running" + country)
    result = sm.OLS(MergedFile[["IntRate"]], sm.add_constant(MergedFile[["TerrIdx","IncidentLag_5", "Lab", "GDPpc", "EODB", "DomCrdt","FixedCapForm","CurStd"]]), missing = "drop").fit()

    wtest = het_white(result.resid, result.model.exog)
    labels = ['Lagrange Multiplier statistic:', 'LM test\'s p-value:', 'F-statistic:', 'F-test\'s p-value:']
    f = open(country+".txt","w+")
    f.write(str(result.summary()))
    f.write(str("\n\nWhite's Test\n"))
    f.write(str(lzip(labels, wtest)))
    f.write(str("\n\n2nd order autocorrelation\n"))
    f.write(str(bg(result, nlags=2)))
    f.write(str("\n\n3rd order autocorrelation\n"))
    f.write(str(bg(result, nlags=3)))
    f.close()

RunningAustralia
RunningBolivia
RunningCanada
RunningDominican Republic
RunningGuatemala
RunningItaly
RunningJapan
RunningJordan
RunningLebanon
RunningMexico
RunningNetherlands
RunningNicaragua
RunningParaguay
RunningPhilippines
RunningUnited Kingdom
RunningUnited States
RunningCosta Rica
RunningSweden
RunningZambia
RunningThailand
RunningHaiti
RunningHonduras
RunningMalaysia
RunningNew Zealand
RunningPeru
RunningJamaica
RunningSouth Africa
RunningKenya
RunningIndonesia
RunningGuyana
RunningLesotho
RunningUganda
RunningGuinea
RunningAlbania
RunningBulgaria
RunningIceland
RunningHungary
RunningArmenia
RunningAzerbaijan
RunningCroatia
RunningUkraine
RunningVietnam
RunningBelarus
RunningCzech Republic
RunningUruguay
RunningSingapore
RunningMauritania
RunningRomania
RunningTajikistan
RunningPakistan
RunningBotswana
RunningGeorgia
RunningSwitzerland
RunningArgentina
RunningNorway
RunningSierra Leone


In [63]:
lags = 6

**Take the bloody shot!**

Y = intercept + α(Incident t) + α1(Incident t-1) + α2(Incident t-2) + α3(Incident t-3) + α4(Incident t-4) + α5(Incident t-5) + α6(Incident t-6) + αc1(Control Labour Force%) + ac2(Control GDPpc) + ac3(Control EODB) + ac4(Control Domestic Crdt Sprd) + ac5(Control Inflation) +ac6(Control Interest Rate) 

New Version

In [85]:
countries = set(Ind_Terr["Country Name"].unique()).intersection(Control_Matrix["Country Name"].unique()).intersection(Dp_FDI["Country Name"].unique())
for country in countries:
    print("Running", country)
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    #Filling NaN Values with mean()
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    cols = []
    for lag in range(1, lags + 1): 
        col = f"IncidentLag_{lag}"
        Ter[col] = Ter["Incidents"].shift(lag)
        cols.append(col)
    Ter.dropna(inplace = True)
    Dp = Dp_FDI[Dp_FDI["Country Name"] == country]
    Ind = pd.merge(Ter, Cntrl,  how='outer', left_on=["Country Name","Year"], right_on = ["Country Name","Year"])
    Dp.fillna(0, inplace = True)
    Ind.fillna(0, inplace = True)
    result = sm.OLS(Dp[["FDI"]].reset_index(drop=True), sm.add_constant(Ind[["Incidents","IncidentLag_1", "IncidentLag_2", "IncidentLag_3", "IncidentLag_4", "IncidentLag_5", "IncidentLag_6","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate","CurStd"]])).fit()
    f= open(country+".txt","w+")
    f.write(str(result.summary()))
    f.close()
    print(result.summary())

Running Switzerland
                            OLS Regression Results                            
Dep. Variable:                    FDI   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     3.833
Date:                Sun, 24 Oct 2021   Prob (F-statistic):             0.0729
Time:                        03:00:09   Log-Likelihood:                -47.551
No. Observations:                  20   AIC:                             125.1
Df Residuals:                       5   BIC:                             140.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           419.3816  

In [ ]:
#Run for specific Country
countries = ["Japan"]
for country in countries:
    print("Running", country)
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    #Filling NaN Values with mean()
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    cols = []
    for lag in range(1, lags + 1): 
        col = f"IncidentLag_{lag}"
        Ter[col] = Ter["Incidents"].shift(lag)
        cols.append(col)
    Ter.dropna(inplace = True)
    Dp = Dp_FDI[Dp_FDI["Country Name"] == country]
    Ind = pd.merge(Ter, Cntrl,  how='outer', left_on=["Country Name","Year"], right_on = ["Country Name","Year"])
    Dp.fillna(0, inplace = True)
    Ind.fillna(0, inplace = True)
    result = sm.OLS(Dp[["FDI"]].reset_index(drop=True), sm.add_constant(Ind[["Incidents","IncidentLag_1", "IncidentLag_2", "IncidentLag_3", "IncidentLag_4", "IncidentLag_5", "IncidentLag_6","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate","CurStd"]])).fit()
    print(result.summary())

Old Below

In [75]:
FinalTable = pd.DataFrame(index = range(0,len(countries)))
FinalTable["Country"] = list(countries)
FinalTable["intercept"] = None
FinalTable["a"] = None
FinalTable["a1"] = None
FinalTable["a2"] = None
FinalTable["a3"] = None
FinalTable["a4"] = None
FinalTable["a5"] = None
FinalTable["a6"] = None
FinalTable["ac1"] = None
FinalTable["ac2"] = None
FinalTable["ac3"] = None
FinalTable["ac4"] = None
FinalTable["ac5"] = None
FinalTable["ac6"] = None

In [76]:
for country in countries:
    print("Running", country)
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    #Filling NaN Values with mean()
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    cols = []
    for lag in range(1, lags + 1): 
        col = f"IncidentLag_{lag}"
        Ter[col] = Ter["Incidents"].shift(lag)
        cols.append(col)
    Ter.dropna(inplace = True)
    Dp = Dp_FDI[Dp_FDI["Country Name"] == country]
    Ind = pd.merge(Ter, Cntrl,  how='outer', left_on=["Country Name","Year"], right_on = ["Country Name","Year"])
    Dp.fillna(0, inplace = True)
    Ind.fillna(0, inplace = True)
    #print(len(Cntrl))
    #print(len(Ind))
    #print(len(Dp))
    Model = LinearRegression().fit(Ind[["IncidentLag_1", "IncidentLag_2", "IncidentLag_3", "IncidentLag_4", "IncidentLag_5"
                                        , "IncidentLag_6","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]],Dp[["FDI"]])
    #print(country, "Model Coefficients:", Model.coef_ , "Model Intercept:", Model.intercept_)
    FinalTable["intercept"][FinalTable["Country"]==country] = Model.intercept_
    FinalTable["a"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["a1"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["a2"][FinalTable["Country"]==country] = Model.coef_[0,1]
    FinalTable["a3"][FinalTable["Country"]==country] = Model.coef_[0,2]
    FinalTable["a4"][FinalTable["Country"]==country] = Model.coef_[0,3]
    FinalTable["a5"][FinalTable["Country"]==country] = Model.coef_[0,4]
    FinalTable["a6"][FinalTable["Country"]==country] = Model.coef_[0,5]
    FinalTable["ac1"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac2"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac3"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac4"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac5"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac6"][FinalTable["Country"]==country] = Model.coef_[0,0]

Running Japan


In [84]:
FinalTable.to_csv("Regression_Output_v1.0.csv", index = False)

**Where the hell have you been? Enjoying death.**